In [1]:
from pyspark.sql import SparkSession, Row
from pyspark.sql import functions as F
import matplotlib.pyplot as plt
from pyspark.sql.functions import avg, count, stddev, max, min
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler
from pyspark.ml import Pipeline
import pandas as pd

In [2]:
#Spark Sesssion Creation
spark = SparkSession.builder \
    .appName("SparkReadOperations") \
    .config("spark.executor.cores", "12") \
    .config("spark.executor.memory", "48g") \
    .config("spark.driver.memory", "8g") \
    .config("spark.sql.shuffle.partitions", "12") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

In [3]:
#Extract
df = spark.read.csv("data/Training-Set.csv", header=True, inferSchema=True)

In [4]:
#Checking the rows and columns
df_dim=(df.count(), len(df.columns))
print(df_dim)

(3116945, 30)


In [5]:
# #Selecting 20K rows intially
# df= df.orderBy(F.rand()).limit(200000)
# df.show(5)

In [6]:
#shows the columns name and data types
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- class: integer (nullable = true)
 |-- cap-diameter: double (nullable = true)
 |-- cap-shape: string (nullable = true)
 |-- cap-surface: string (nullable = true)
 |-- cap-color: string (nullable = true)
 |-- does-bruise-or-bleed: string (nullable = true)
 |-- gill-attachment: string (nullable = true)
 |-- gill-spacing: string (nullable = true)
 |-- gill-color: string (nullable = true)
 |-- stem-height: double (nullable = true)
 |-- stem-width: double (nullable = true)
 |-- stem-root: string (nullable = true)
 |-- stem-surface: string (nullable = true)
 |-- stem-color: string (nullable = true)
 |-- veil-type: string (nullable = true)
 |-- veil-color: string (nullable = true)
 |-- has-ring: string (nullable = true)
 |-- ring-type: string (nullable = true)
 |-- spore-print-color: string (nullable = true)
 |-- habitat: string (nullable = true)
 |-- season: string (nullable = true)
 |-- log-cap-diameter: double (nullable = true)
 |-- log-stem-heig

In [7]:
#Replacing the Nul/-na string with empty cells
df = df.replace(["NULL", "_NA_","?"], None)
df.show()

+---+-----+------------+---------+-----------+---------+--------------------+---------------+------------+----------+-----------+----------+---------+------------+----------+---------+----------+--------+---------+-----------------+-------+------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+
| id|class|cap-diameter|cap-shape|cap-surface|cap-color|does-bruise-or-bleed|gill-attachment|gill-spacing|gill-color|stem-height|stem-width|stem-root|stem-surface|stem-color|veil-type|veil-color|has-ring|ring-type|spore-print-color|habitat|season|  log-cap-diameter|   log-stem-height|    log-stem-width|         Stem_Area|       Stem_Volume|          Cap_Area|     Stem_Cap_Ratio|         Stem_Shape|
+---+-----+------------+---------+-----------+---------+--------------------+---------------+------------+----------+-----------+----------+---------+------------+----------+---------+----------+-

In [8]:
#Cheking the null values
schema_info = {field.name: field.dataType.simpleString() for field in df.schema.fields}
spark.createDataFrame(
    [
        Row(
            column=c,
            data_type=schema_info[c],
            null_count=df.filter(F.col(c).isNull()).count(),
            )
        for c in df.columns
    ]

).withColumn(
    "null_ratio", F.bround(F.col("null_count") * 100 / F.lit(df.count()))
).show(30)

Py4JJavaError: An error occurred while calling o158.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 99.0 failed 1 times, most recent failure: Lost task 0.0 in stage 99.0 (TID 430) (LAPTOP-BDG5LM2U executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:331)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadChec

In [ ]:
df.select([F.col(c) for c in df.columns]).summary("count").show()


Py4JJavaError: An error occurred while calling o311.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 196.0 failed 1 times, most recent failure: Lost task 0.0 in stage 196.0 (TID 847) (LAPTOP-BDG5LM2U executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:331)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCh

#Feature Selection

In [ ]:
#Since some columns empty cells percantage is high
#and some columns contains similar types of data like log-cap diameter is the transformed form of cap-diameter
#deleting those features
columns_to_drop = ["id","cap-diameter","stem-height","stem-width", "stem-root", "gill-attachment","stem-surface", "veil-type",
                   "veil-color", "spore-print-color", "gill-spacing", "cap-surface"]

df_cleaned = df.drop(*columns_to_drop)

In [ ]:
df_cleaned.show()

+-----+---------+---------+--------------------+----------+----------+--------+---------+-------+------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+
|class|cap-shape|cap-color|does-bruise-or-bleed|gill-color|stem-color|has-ring|ring-type|habitat|season|  log-cap-diameter|   log-stem-height|    log-stem-width|         Stem_Area|       Stem_Volume|          Cap_Area|     Stem_Cap_Ratio|         Stem_Shape|
+-----+---------+---------+--------------------+----------+----------+--------+---------+-------+------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+
|    0|        f|    Other|                   f|         w|         w|       f|        f|      d|     a|2.2823823856765264| 1.706564623164823| 2.796671392755738|  4.25431967562815| 6.733359809474644| 4.124246894018546| 0.41

In [ ]:
#Columns
df_cleaned.printSchema()

root
 |-- class: integer (nullable = true)
 |-- cap-shape: string (nullable = true)
 |-- cap-color: string (nullable = true)
 |-- does-bruise-or-bleed: string (nullable = true)
 |-- gill-color: string (nullable = true)
 |-- stem-color: string (nullable = true)
 |-- has-ring: string (nullable = true)
 |-- ring-type: string (nullable = true)
 |-- habitat: string (nullable = true)
 |-- season: string (nullable = true)
 |-- log-cap-diameter: double (nullable = true)
 |-- log-stem-height: double (nullable = true)
 |-- log-stem-width: double (nullable = true)
 |-- Stem_Area: double (nullable = true)
 |-- Stem_Volume: double (nullable = true)
 |-- Cap_Area: double (nullable = true)
 |-- Stem_Cap_Ratio: double (nullable = true)
 |-- Stem_Shape: string (nullable = true)



In [ ]:
#Drop null values
df_cleaned = df_cleaned.dropna()
df_cleaned.show()

+-----+---------+---------+--------------------+----------+----------+--------+---------+-------+------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+
|class|cap-shape|cap-color|does-bruise-or-bleed|gill-color|stem-color|has-ring|ring-type|habitat|season|  log-cap-diameter|   log-stem-height|    log-stem-width|         Stem_Area|       Stem_Volume|          Cap_Area|     Stem_Cap_Ratio|         Stem_Shape|
+-----+---------+---------+--------------------+----------+----------+--------+---------+-------+------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+
|    0|        f|    Other|                   f|         w|         w|       f|        f|      d|     a|2.2823823856765264| 1.706564623164823| 2.796671392755738|  4.25431967562815| 6.733359809474644| 4.124246894018546| 0.41

In [ ]:
#Drop Duplicate values
df_cleaned = df_cleaned.dropDuplicates()

In [ ]:
df_dim=(df_cleaned.count(), len(df_cleaned.columns))
print(df_dim)

(3113962, 18)


In [ ]:
total_row_count = df_cleaned.count()

schema_info = {field.name: field.dataType.simpleString() for field in df_cleaned.schema.fields}

summary_df = spark.createDataFrame([
    Row(
        column=c,
        data_type=schema_info[c],
        other_count=df_cleaned.filter(F.col(c) == "Other").count()
    )
    for c in df_cleaned.columns
])


summary_df = summary_df.withColumn(
    "other_ratio", F.bround(F.col("other_count") * 100 / F.lit(total_row_count), 2)
)

summary_df.show(30)


Py4JJavaError: An error occurred while calling o403.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 284.0 failed 1 times, most recent failure: Lost task 0.0 in stage 284.0 (TID 1124) (LAPTOP-BDG5LM2U executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:331)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadC

In [ ]:
df_cleaned.describe().show()

+-------+------------------+---------+---------+--------------------+----------+----------+--------+---------+-------+-------+--------------------+------------------+------------------+------------------+------------------+--------------------+-------------------+----------+
|summary|             class|cap-shape|cap-color|does-bruise-or-bleed|gill-color|stem-color|has-ring|ring-type|habitat| season|    log-cap-diameter|   log-stem-height|    log-stem-width|         Stem_Area|       Stem_Volume|            Cap_Area|     Stem_Cap_Ratio|Stem_Shape|
+-------+------------------+---------+---------+--------------------+----------+----------+--------+---------+-------+-------+--------------------+------------------+------------------+------------------+------------------+--------------------+-------------------+----------+
|  count|           3113962|  3113962|  3113962|             3113962|   3113962|   3113962| 3113962|  3113962|3113962|3113962|             3113962|           3113962|      

In [ ]:

# List of categorical columns
categorical_columns = [
    "cap-shape", "cap-color", "does-bruise-or-bleed",
    "gill-color", "has-ring", "ring-type", "habitat", "season"
]

# Compute unique values count for each categorical column
unique_counts = {col_name: df_cleaned.select(col_name).distinct().count() for col_name in categorical_columns}

# Display results
for col_name, count in unique_counts.items():
    print(f"{col_name}: {count} unique values")

cap-shape: 5 unique values
cap-color: 7 unique values
does-bruise-or-bleed: 3 unique values
gill-color: 7 unique values
has-ring: 3 unique values
ring-type: 2 unique values
habitat: 4 unique values
season: 4 unique values


In [ ]:
df_cleaned.printSchema()

root
 |-- class: integer (nullable = true)
 |-- cap-shape: string (nullable = true)
 |-- cap-color: string (nullable = true)
 |-- does-bruise-or-bleed: string (nullable = true)
 |-- gill-color: string (nullable = true)
 |-- stem-color: string (nullable = true)
 |-- has-ring: string (nullable = true)
 |-- ring-type: string (nullable = true)
 |-- habitat: string (nullable = true)
 |-- season: string (nullable = true)
 |-- log-cap-diameter: double (nullable = true)
 |-- log-stem-height: double (nullable = true)
 |-- log-stem-width: double (nullable = true)
 |-- Stem_Area: double (nullable = true)
 |-- Stem_Volume: double (nullable = true)
 |-- Cap_Area: double (nullable = true)
 |-- Stem_Cap_Ratio: double (nullable = true)
 |-- Stem_Shape: string (nullable = true)



In [ ]:
df_dim=(df_cleaned.count(), len(df_cleaned.columns))
print(df_dim)

(3113962, 18)


In [ ]:
# Convert to Pandas DataFrame
df_cleaned_pd = df_cleaned.toPandas()

# Save as a single CSV file
df_cleaned_pd.to_csv("data/powerBI.csv", index=False)

PermissionError: [Errno 13] Permission denied: 'data/powerBI.csv'

In [ ]:
# Save as a single CSV file using PySpark (Avoids Memory Issues)
df_cleaned.coalesce(1).write.mode("overwrite").option("header", "true").csv("data/powerBI")

# Move the CSV out of the folder (Optional)
import shutil
import glob

csv_file = glob.glob("data/powerBI/*.csv")[0]  # Get the actual CSV file
shutil.move(csv_file, "data/powerBI.csv")  # Move it out of the folder
shutil.rmtree("data/powerBI")  # Remove extra folder


Py4JJavaError: An error occurred while calling o623.csv.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:

In [ ]:
# categorical_columns = [c for c, dtype in df_cleaned.dtypes if dtype == "string"]
# for col_name in categorical_columns:

#     category_counts = df.groupBy(col_name).count().orderBy(F.desc("count")).toPandas()

#     # Plot bar chart
#     plt.figure(figsize=(6, 4))
#     plt.bar(category_counts[col_name], category_counts["count"], color='skyblue')
#     plt.xlabel(col_name)
#     plt.ylabel("Count")
#     plt.title(f"Distribution of {col_name}")
#     plt.xticks(rotation=45)
#     plt.show()

#Outliers

In [ ]:
# List of categorical and numerical columns
numerical_cols = ["log-cap-diameter", "log-stem-height", "log-stem-width",
                  "Stem_Area", "Stem_Volume", "Cap_Area", "Stem_Cap_Ratio"]
categorical_cols = ["cap-shape", "cap-color", "does-bruise-or-bleed", "gill-color",
                    "stem-color", "has-ring", "ring-type", "habitat", "season", "Stem_Shape"]

In [ ]:
# Handle Outliers in Numerical Columns using Z-score
mean_stddev = df_cleaned.select([F.mean(col).alias(col + "_mean") for col in numerical_cols] +
                               [F.stddev(col).alias(col + "_stddev") for col in numerical_cols]).first()

# Calculate Z-score for each numerical column
z_scores = []
for col in numerical_cols:
    mean = mean_stddev[col + "_mean"]
    stddev = mean_stddev[col + "_stddev"]
    z_score_column = (F.col(col) - mean) / stddev
    z_scores.append(z_score_column.alias(col + "_z_score"))

# Add Z-scores to the DataFrame
df_with_z_scores = df_cleaned.select("*").select(*df_cleaned.columns, *z_scores)

# Filter out rows where Z-score > 3 or < -3
df_no_outliers_numerical = df_with_z_scores.filter(
    (F.abs(F.col("log-cap-diameter_z_score")) <= 3) &
    (F.abs(F.col("log-stem-height_z_score")) <= 3) &
    (F.abs(F.col("log-stem-width_z_score")) <= 3) &
    (F.abs(F.col("Stem_Area_z_score")) <= 3) &
    (F.abs(F.col("Stem_Volume_z_score")) <= 3) &
    (F.abs(F.col("Cap_Area_z_score")) <= 3) &
    (F.abs(F.col("Stem_Cap_Ratio_z_score")) <= 3)
)

In [ ]:
# Handle Outliers in Categorical Columns
# Identify low frequency categories and treat them as outliers
for col in categorical_cols:
    # Get the frequency of each category
    category_count = df_cleaned.groupBy(col).count().withColumnRenamed("count", f"{col}_count")
    total_count = df_cleaned.count()

    # Calculate frequency percentage
    category_freq = category_count.withColumn(f"{col}_percent", (F.col(f"{col}_count") / total_count) * 100)

    # Treat categories with less than 5% frequency as outliers
    low_freq_categories = category_freq.filter(F.col(f"{col}_percent") < 5).select(col).distinct().rdd.flatMap(lambda x: x).collect()

    # Replace low frequency categories with "Other"
    if low_freq_categories:
        df_no_outliers_numerical = df_no_outliers_numerical.withColumn(
            col,
            F.when(F.col(col).isin(low_freq_categories), F.lit("Other")).otherwise(F.col(col))
        )

# Show the number of rows before and after handling outliers
print(f"Rows before removing outliers: {df_cleaned.count()}")
print(f"Rows after removing outliers: {df_no_outliers_numerical.count()}")


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 362.0 failed 1 times, most recent failure: Lost task 0.0 in stage 362.0 (TID 1361) (LAPTOP-BDG5LM2U executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:331)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:93)
	at org.a

In [ ]:
# Convert to Pandas DataFrame
df_no_outliers_numerical_pd = df_no_outliers_numerical.toPandas()

# Save as a single CSV file
df_no_outliers_numerical_pd.to_csv("data/powerBI.csv", index=False)

PermissionError: [Errno 13] Permission denied: 'data/powerBI.csv'

#Feature Engineering

In [ ]:

# Encode Categorical Variables
indexers = [StringIndexer(inputCol=col, outputCol=col+"_index", handleInvalid="keep") for col in categorical_cols]
encoder = OneHotEncoder(inputCols=[col+"_index" for col in categorical_cols],
                        outputCols=[col+"_encoded" for col in categorical_cols])

# Assemble Features into Vectors
assembler = VectorAssembler(inputCols=[col+"_encoded" for col in categorical_cols] + numerical_cols,
                            outputCol="features")

# Scale Numerical Features
scaler = StandardScaler(inputCol="features", outputCol="scaled_features")

# Apply Feature Engineering Pipeline
pipeline = Pipeline(stages=indexers + [encoder, assembler, scaler])
df_transformed = pipeline.fit(df_no_outliers_numerical).transform(df_no_outliers_numerical)

# Show Transformed Data
df_transformed.select("class", "scaled_features").show(truncate=False)

In [ ]:


# Convert to Pandas DataFrame
df_transformed_pd = df_transformed.select("class", "scaled_features").toPandas()

# Save as a single CSV file
df_transformed_pd.to_csv("data/clean.csv", index=False)
